<a href="https://colab.research.google.com/github/MyatKaung/freecodecampMachineLearning/blob/main/fcc_book_recommendation_knn_completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-04-22 10:19:24--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  7.32MB/s    in 4.1s    

2023-04-22 10:19:28 (6.05 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# Preprocessing
# Filter users who have rated more than 200 books
user_ratings_count = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_ratings_count[user_ratings_count >= 200].index)]

# Filter books that have more than 100 ratings
book_ratings_count = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_ratings_count[book_ratings_count >= 100].index)]

# Merge the books and ratings dataset
df_merged = pd.merge(df_ratings, df_books, on='isbn')

# Create a pivot table
book_matrix = df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Convert the pivot table to a sparse matrix
book_sparse_matrix = csr_matrix(book_matrix.values)

# Fit the KNN model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(book_sparse_matrix)



NearestNeighbors(algorithm='brute', metric='cosine')

In [7]:
# function to return recommended books - this will be tested
# Recommendation function
def get_recommends(book=''):
    if book not in book_matrix.index:
        return None

    book_idx = book_matrix.index.get_loc(book)
    distances, indices = knn.kneighbors(book_matrix.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6)

    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_books.append([book_matrix.index[indices.flatten()[i]], distances.flatten()[i]])

    return recommended_books
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)




[['The Lovely Bones: A Novel', 0.7234864], ["The Pilot's Wife : A Novel", 0.81926787], ['The Joy Luck Club', 0.81986046], ['The Notebook', 0.82366824], ['Bel Canto: A Novel', 0.8247875]]
